In [1]:
import torch
import os
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainingArguments,
    set_seed,
)

/mnt/cache/xuekui/opt/miniconda3/envs/torch2.0/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name_or_path = "/mnt/petrelfs/xuekui/pretrain_weights/nlp/DoctorGLM/pretrain_model"
ptuning_checkpoint = "/mnt/petrelfs/xuekui/pretrain_weights/nlp/DoctorGLM/ptuningv2/pytorch_model.bin"
pre_seq_len = 128


config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
config.pre_seq_len = pre_seq_len
config.prefix_projection = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [5]:
model = AutoModel.from_pretrained(model_name_or_path, config=config, trust_remote_code=True)
prefix_state_dict = torch.load(ptuning_checkpoint, map_location="cpu")

new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    if k.startswith("transformer.prefix_encoder."):
        new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Loading checkpoint shards: 100%|██████████| 8/8 [00:16<00:00,  2.07s/it]
Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at /mnt/petrelfs/xuekui/pretrain_weights/nlp/DoctorGLM/pretrain_model and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [6]:
model = model.half()
model.transformer.prefix_encoder.float()
model = model.eval()

In [7]:
model

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (word_embeddings): Embedding(130528, 4096)
    (layers): ModuleList(
      (0-27): 28 x GLMBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): SelfAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): GLU(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (final_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
    (prefix_encoder): PrefixEncoder(
      (embedding): Embedding(128, 229376)
    )
    (dropout): Dropout(p=0.1, inplace=Fal

In [10]:
tokenizer("你是谁").input_ids

[5, 108293, 130001, 130004]